# SAR-GE

SAR-GE is a Synthetic Aperture Radar payload carrying satellite that aims to detect Greenhouse Emissions from pipelines across Canada.
This is a Nadir pointer, no question.

Ch. 14 SME-SMAD recommends block redundancy for 5yr lifetime. What does this actually mean? Nothing at this stage.

TDRS may be reference (sme pg404)

In [1]:
using Unitful
using Dates
using UnitfulData

𝑐 = 299_792_458u"m/s"
k = 1.380_649e-23u"J/K"
Design_Lifetime = 5u"yr"

5 yr

## Orbit
The orbit was designed using STK's orbit wizard - very convenient.

In [2]:
# Orbital Parameters
P = 86_170.5u"s";
e = 0;
i = 121u"°"; # Retrograde (i > 90°)
ω = 0;
Ω = 50u"°";
ν = 30u"°";

# Earth Parameters
μ = 3.986_004_418e14u"m^3*s^-2";
R_terra = 6_378.136_6u"km"; # SME-SMAD equatorial
J₂ = 0.001_082_635_9; # SME-SMAD

0.0010826359

SSE pg. 81 states that at high altitudes (such as in GEO), the gravitational pull of Luna and Sol become significant, as well as radiation pressure from Sol.

For circuluar orbits ($e=0$) $a=r$ and $P=2\pi\sqrt{a^3/\mu}$

In [39]:
# Orbit Radius and altitude
r = u"km"(∛((P/(2π))^2*μ));
h = r - R_terra # matches STK
n = u"d^-1"(1/P) # revolutions per day
h

35788.12397940338 km

In [4]:
# Vis viva to find total energy per unit mass and velocity
ε = u"J/kg"(-μ/(2*r))
V_c = u"m/s"(√(μ/r))

3074.583864909837 m s^-1

In [5]:
# Average values of Third Body Interaction (SME-SMAD)
DΩ_luna = -0.003_38u"°/d^2" * (cos(i)/n) # to make the units work, per SME-SMAD)
DΩ_sol = -0.001_54u"°/d^2" * (cos(i)/n) # "
Dω_luna = 0.001_69u"°/d^2" * (4u"d" - 5 * sin(i)^2/n); # "
Dω_sol = 0.000_77u"°/d^2" * (4u"d" - 5 * sin(i)^2/n); # "

In [6]:
# Solar radiation pressure (SME-SMAD)
#aᵣ = -4.5e-6 A/m [m/s^2] where A is projected cross-sectional area of sat. wrt. sol in m^2, m is mass of sat. in kg.
# per SME-SMAD pg. 214, above 800km altitude the acceleration from sol becomes an important perturbing force, frequently dominant wrt. other perturbations.'
# SSE too
# SSE Table 4.2 pg94 gives mags of acceleration
# Sun (mean) 3.5e-6
# Moon (mean) 7.3e-6
# Jupiter (max) 5.2e-11 all m/s^2

In [7]:
# Let's neglect atmospheric drag, as we are in GEO

## Communications
We'll first calculate the bulk data rate using the area of Canada proper and the orbital period. The data groups are given in the project specifications.

In [8]:
# Physical Parameters
A_Canada = 8.138_81e6u"km^2" + 203_109u"km^2" # Contiguous Canada + Maritimes

8.341919e6 km^2

### Payload Data

In [9]:
# Prescribed Data
Payload_Resolution = 5u"m"
Image_Area = (100u"km")^2
Swath_Width = 100u"km"
Max_TTI = 14u"d" # TTI - Time to Image
Pixel_Size = 16u"bit" # Could be 32 per project document, but let's be conservative

16 bits

In [50]:
# Image Size
Image_Pixels = u"m/m"(Image_Area / Payload_Resolution^2)
Image_Size = u"MByte"(Pixel_Size * Image_Pixels)

# Imaging Canada
Images_per_Canada = A_Canada / Image_Area
Images_per_Canada *= 1.04 # Suggested to account for 4% swath overlap
# technically since the swaths only overlap in one dimension, i think this should be 1.02.
Canada_Size = u"GByte"(Images_per_Canada * Image_Size)

# Bulk payload data rate
Avg_Payload_Data_Rate = u"kBps"(Canada_Size / Max_TTI)

573.7827883597884 kBps

### Housekeeping Data

In [11]:
# Prescribed Data
Error_Detection_Rate = 3u"bit/Byte"
Latitude = 32u"bit" # per image
Longitude = 32u"bit" # per image
Altitude = 32u"bit" # per image
Time = 64u"bit" # per image
Synchronization = 24u"bit" # per image
Health_Data_Rate = u"bps"(100u"minute^-1" * 16u"bit")

80//3 bps

In [53]:
# Housekeeping Data per image
Housekeeping_per_Image = u"MByte"(Image_Size * Error_Detection_Rate + 
    (Latitude + Longitude + Altitude + Time + Synchronization))
Housekeeping_Size = u"GByte"(Housekeeping_per_Image * Images_per_Canada) # per Canada imaged

Avg_Housekeeping_Data_Rate = u"kBps"(Housekeeping_Size / Max_TTI + Health_Data_Rate)

215.17189546450913 kBps

In [13]:
# Total Average Data Rate
Avg_Data_Rate = Avg_Payload_Data_Rate + Avg_Housekeeping_Data_Rate
# this will be weighted by the ground-station access time ratio to find the real transmit rate needed

788.9546838242975 kBps

## Access Time
Fun annulus math!

In [55]:
# Approximation of arc angle subtended by a 10° elevation requirement
# The oblateness of Earth makes this a bit more complicated, but oh well. Margins.
δ = 10u"°"
d = R_terra * (√((r/R_terra)^2 - cos(δ)^2) - sin(δ))
γ = asin(d * sin(90u"°" + δ) / r) # Half-angle, matches geogebra
Access_Arc = r * 2γ
Total_Arc = r * 2π
Access_Ratio = Access_Arc / Total_Arc

Access_Time = P * Access_Ratio
u"hr"(Access_Time)

9.499117967294225 hr

In [45]:
# Slides say to allocate 20% of time for uplink
Uplink_Time = Access_Time * 0.20
Downlink_Time = Access_Time - Uplink_Time

# Real Downlink Data Rate
Downlink_Data_Rate = u"MBps"(Avg_Data_Rate / (Downlink_Time / P))
print("Peak Downlink Data Rate: $(round(u"MBps",Downlink_Data_Rate,sigdigits=4)) for $(round(u"hr",Downlink_Time,sigdigits=3)) every $(round(u"hr",P,sigdigits=4))")
u"bit/s"(Downlink_Data_Rate)
# Uplink
# Rounding error, essentially.

Peak Downlink Data Rate: 2.485 MBps for 7.6 hr every 23.94 hr

1.9880389543228846e7 bit s^-1

Using the built-in default access parameters for the Gatineau ground station in STK, we get less access time and thus more peak data rate requirements. For preliminary design, let's just go with the less stringent requirement. Maybe in the future I can just shove the shorter access time in and get better.

## Subsystem and Link Budget
~~e are all minimum bit error rate on this blessed day.~~
Refer to the spreadsheet :/

In [16]:
# Prescribed Data
Maximum_Telemetry_BER = 10e-5; # 6TT&C, s.26
Maximum_Telecommand_BER = 10e-7; # ^
Gatineau_Dish_Diameter = 13u"m"; # https://natural-resources.canada.ca/science-data/science-research/research-centres/gatineau-satellite-station
Pointing_Error = 1u"°"
Downlink_Center_Frequency = 8.2u"GHz"; # X-band Earth sciences, SMESMADpg631
Max_Bandwidth = 400u"MHz"; # ^
λ = 𝑐 / Downlink_Center_Frequency
#G_r = uconvertrp(u"dB",20*log10(ustrip(u"MHz"(Downlink_Center_Frequency))) + 20*log10(ustrip(u"ft"(Gatineau_Dish_Diameter))) - 52.6) # Antenna gain, parabolic, 54% efficiency, ELEC4509 Course Pack II.37 -- Nah, this seems off
# !!Q is this rp or p? 35.3 or 17.6
G_r = (10log10(0.54) + 20log10(π) + 20log10(ustrip(Gatineau_Dish_Diameter)) - 20log10(ustrip(u"m"(𝑐 / Downlink_Center_Frequency))))u"dB" # congruent with firesatII example in SMESMAD pg635
# Error correction for payload data is prescribed and thus out of our hands.

58.285665087364876 dB

~~To make things easier (more complicated), we're going to go with a QPSK Viterbi scheme with R=1/2 and K=8. From graph 6TT&Cs.32, at BER=1e-5, $E_b/N_0=3.7dB$. And from slide 40, $\left(\frac CN\right)_\text{required}=R\frac{E_b}{N_0}$, right?~~ uhhhh.

We're going to keep things simple for now.

$(\frac{C}{N})_\text{required}=\frac RB\ln{\frac{1}{2\,\text{BER}}}$

$B=0.6R$ for QPSK (6TT&S.s23)

$(\frac CN)_\text{req'd}=\frac{1}{0.6}\ln{\frac{1}{2\,\text{BER}}}$

In [44]:
#C_N_required = Downlink_Data_Rate * 3.7u"J/bit" NOPE FUCK IT
C_N_required = uconvertp(u"dB",0.6^-1 * log(2Maximum_Telemetry_BER^-1))
println("C/N Required: $(C_N_required)")

Bandwidth = 0.6u"1/bit" * Downlink_Data_Rate
println("Bandwidth: $(round(u"MHz",Bandwidth,sigdigits=5)), within X-band? $(Bandwidth < Max_Bandwidth), FBW = $(round((Bandwidth / Max_Bandwidth |> upreferred) * 100))%")

# Link Equation/Budget


# Lets try a new approach... For Intelsat approval, G/T ≥ 40.7dB + 20log(f/4) and G ≥ 57dB + 20log(f/4) 4509 course pack
Min_FOM = 40.7u"dB" + 20u"dB"*log10(ustrip(u"GHz"(Downlink_Center_Frequency))/4)
Min_RxG = 57u"dB" + 20u"dB"*log10(ustrip(u"GHz"(Downlink_Center_Frequency))/4)
println("G_r < Min G_r: $(G_r < Min_RxG) so we'll use $Min_RxG. G/T = $Min_FOM")
G_r = Min_RxG
L_rx_line = 2u"dB" #assume

# Tx Antenna Specs
Tx_Diameter = 0.5u"m"
Tx_η = 0.50
G_t = (-159.59 + 20log10(ustrip(u"m"(Tx_Diameter))) + 20log10(ustrip(u"Hz"(Downlink_Center_Frequency))) + 10log10(Tx_η))u"dB"
println("G_t: $G_t") # agrees with fig 16-10 sme yay
L_tx_line = 3u"dB" #assume


# Transmitter power det'd by the power budget. say 150W of 170W?
P_t = u"dBm"(150u"W")

EIRP = G_t * P_t - L_tx_line # about right comp. to. 16-13 p480 sme
println("EIRP: $EIRP")

# Path loss
#L_path = uconvertp(u"dB", ((4π*d)/λ)^2 |> upreferred) # yes
L_path = 92.46u"dB" + 20u"dB"*log10(ustrip(u"km"(h))) + 20u"dB"*log10(ustrip(u"GHz"(Downlink_Center_Frequency))) #sme form eq16-21
println("Path loss: $L_path") # agrees with sme
L_prec = 7u"dB" #ballpark assumed

# Since P_r = C, we'll use the standard 290K for the temperature
# kTB
#Noise = u"dBm"(k * 290u"K" * Bandwidth)
Rx_Noise_Temp = 70u"K" # cryo cooled
Noise = (10log10(ustrip(k)) + 10log10(ustrip(Rx_Noise_Temp)) + 10log10(ustrip(u"Hz"(Max_Bandwidth))))*1u"dB"
println("Noise Power: $Noise")

# Pointing loss
# Good assumption for ADCS pointing is 1° - could use for θ3dB
θ_3dB = 70u"°"*λ/Tx_Diameter |> u"°"
L_point = uconvertp(u"dB", 12 * (Pointing_Error / θ_3dB)^2)
println("θ_3dB = $θ_3dB = Beamwidth and L_point = $L_point ")

# Atmospheric Attenuation
# From Fig16-18sme, the total zenith attenuation at 8GHz is 0.05dB, basically negligible


C_N_achieved = P_t + G_t + G_r - L_path - Noise - L_point
C = EIRP + G_r - L_path - L_prec - L_rx_line - L_point
println("C: $C")
P_r = C
# Bit rate R_b (dB-Hz) = 10log10(R_b(bps))
R_b = (10log10(ustrip(u"bit/s"(Downlink_Data_Rate)))+30)*1u"dBm"
println("R_b = $R_b")
C_N0_down = EIRP + Min_FOM - (L_path + L_prec + L_rx_line + L_point) + 228.6u"dB"
println("C/N_0 = $C_N0_down")
Eb_N0_down = 10log10(C_N0_down / R_b)

# 80K for satellite noise temp
# NO! SC sees mostly deep space at 3K pg 401 sme

# 290K for ground station

# EO database - find similar missions and compare mass/power breakdowns
d
# Pointing loss is based on θ_3dB 

C/N Required: 12.176369096785427 dB
Bandwidth: 11.928 MHz, within X-band? true, FBW = 3.0%
G_r < Min G_r: false so we'll use 63.235077221115084 dB. G/T = 46.93507722111509 dB
G_t: 29.6553771777549 dB
EIRP: 78.41628976831171 dBm
Path loss: 201.8110557113407 dB
Noise Power: -124.12758685979549 dB
θ_3dB = 5.118407819512196° = Beamwidth and L_point = -3.390885263972731 dB 
C: -65.76880345794117 dBm
R_b = 21.1277080587492 dBm
C/N_0 = 146.5311965420588 dBm


40588.24626112903 km

In [18]:
# Sanity/Unitful Check
P_t = 1u"mW"
G_t = 10
L_p = 0.1
EIRP = P_t * G_t * L_p
println("$EIRP = $(u"dBm"(EIRP))")
P_tr = 10log10(1)u"dBm"
G_tr = 10log10(10)u"dB"
L_pr = 10log10(0.1)u"dB"
println(P_tr + G_tr + L_pr)
println(P_tr * G_tr * L_pr)

1.0 mW = 0.0 dBm
dBm 
0.0 dBm


# Stationkeeping and Propulsion
Yay!! Orbit math!!

In [19]:
# "Provided" data
Earth_Obliquity = 23.44u"°"
Lunar_Inclination = 5.14u"°"

# Derived
γ = i - Earth_Obliquity
α = γ - Lunar_Inclination
println("α: $α, γ: $γ")

α: 92.42°, γ: 97.56°


In [20]:
# Stationkeeping ΔV estimations
ΔV_Moon = abs(102.67u"m/s/yr" * cos(α) * sin(α))
ΔV_Sun = abs(40.17u"m/s/yr" * cos(γ) * sin(γ))
NS_StnKeep = (ΔV_Moon + ΔV_Sun) * Design_Lifetime

# SME-SMAD says that GEO represents an "excellent application for electric propulsion for orbit maintenance" pg. 260.

47.85157795730932 m s^-1

In [21]:
# Propulsion
# I will cover GTO->GEO transfer later
# Prelim now I guess. 
# sme says pg253fig10-16 hohmann from 185km alt. of about 4km/s
ΔV_transfer = 4u"km/s"
# M_p = M_f * (exp(ΔV/(I_sp * 𝑔))-1) # Total delta V pg 400 sec14 <- good resource?

# Decomissioning
ΔV_disposal = 18u"m/s"

18 m s^-1

# Power Budget
Preliminary, table14-20 and appA SME

In [22]:
# Prescribed Data
P_Payload = 350u"W"

# Table 14-20 High Earth
PPPayload = 0.33 # changed from 35-33% for structure ↓
PPStructure = 0.02 # we have some tho (so changed from 0 -2%)
PPThermal = 0.14
PPPower = 0.07
PPTTC = 0.16
PPData = 0.10
PPADCS = 0.16
PPPropulsion = 0.02
# avg power 691 W they say

P_Total = P_Payload / PPPayload
println("Payload: $P_Payload")
P_Structure = P_Total * PPStructure
println("Structure: $P_Structure")
P_Thermal = P_Total * PPThermal
println("Thermal: $P_Thermal")
P_Power = P_Total * PPPower
println("Power: $P_Power")
P_TTC = P_Total * PPTTC
println("TT&C: $P_TTC")
P_Data = P_Total * PPData
println("Data: $P_Data")
P_ADCS = P_Total * PPADCS
println("ADCS: $P_ADCS")
P_Propulsion = P_Total * PPPropulsion
println("Propulsion: $P_Propulsion")
println("------------------")
println("Total: $P_Total")

# Bro...
# Since γ = 97.56° we hardly need any mechanism for pointing the solar panels at all
# Just 1DoF for kicks
# Also eclipse time is 0 lmao
# Always sunlight
# Add a battery just as a buffer, maybe the subsystem needs to restart or something, severing the power link between solar array and control


# Required power yada yada
PD_in = 1366u"W/m^2"
η = 18.5e-2 # GaAs achieved ish
#T_cell = 0#°C slide 7.23 says 160C to 90C for GEO
#η = 30 * cos((T_cell + 50)/250)^1.2
θ = 10u"°" # "Worst case"
PD_out = PD_in * η * cos(θ)
println("PD_out: $PD_out")
A_Solar_Reqd = P_Total / PD_out


Payload: 350 W
Structure: 21.21212121212121 W
Thermal: 148.4848484848485 W
Power: 74.24242424242425 W
TT&C: 169.6969696969697 W
Data: 106.06060606060606 W
ADCS: 169.6969696969697 W
Propulsion: 21.21212121212121 W
------------------
Total: 1060.6060606060605 W
PD_out: 248.8707672637151 W m^-2


4.261673929273472 m^2

# Mass Budget
Preliminary, table14-18 and app A SME

In [23]:
# Table 14-18 High Earth
PMPayload = 0.32
PMStructure = 0.24
PMThermal = 0.04
PMPower = 0.17
PMTTC = 0.04
PMData = 0.03
PMADCS = 0.06
PMPropulsion = 0.07
PMOther = 0.03
PMPropellant = 0.72

# Okay
# If payload then
Dry_Mass = 200u"kg" / PMPayload
println("Payload Mass: $(200u"kg")")
# and then
M_Structure = Dry_Mass * PMStructure
println("Structure Mass: $M_Structure")
# etc..
M_Thermal = Dry_Mass * PMThermal
println("Thermal: $M_Thermal")
M_Power = Dry_Mass * PMPower
println("Power: $M_Power")
M_TTC = Dry_Mass * PMTTC
println("TT&C: $M_TTC")
M_Data = Dry_Mass * PMData
println("Data: $M_Data")
M_ADCS = Dry_Mass * PMADCS
println("ADCS: $M_ADCS")
M_Propulsion = Dry_Mass * PMPropulsion
println("Propulsion: $M_Propulsion")
M_Other = Dry_Mass * PMOther
println("Other: $M_Other")
println("-----------------------")
println("Dry Mass: $Dry_Mass")
M_Propellant = Dry_Mass * PMPropellant
println("Propellant: $M_Propellant")
Wet_Mass = Dry_Mass + M_Propellant
println("-----------------------")
println("Wet Mass: $Wet_Mass")


Payload Mass: 200 kg
Structure Mass: 150.0 kg
Thermal: 25.0 kg
Power: 106.25000000000001 kg
TT&C: 25.0 kg
Data: 18.75 kg
ADCS: 37.5 kg
Propulsion: 43.75000000000001 kg
Other: 18.75 kg
-----------------------
Dry Mass: 625.0 kg
Propellant: 450.0 kg
-----------------------
Wet Mass: 1075.0 kg
